In [1079]:
import numpy as np

In [1080]:
import pandas as pd

In [1081]:
df = pd.read_excel("Biertoto.xlsx")

In [1082]:
df = df.dropna(subset=['Heim']).copy()

In [1083]:
new_df = df.copy()


In [1084]:
tordiff = lambda row: abs(row.Htore - row.Gtore)

In [1085]:
tordiff_Uwe = lambda row: abs(row.UHT - row.UGT)

In [1086]:
tordiff_Schadix = lambda row: abs(row.SHT - row.SGT)

In [1087]:
def get_year(row)->int:
    """add 19 to string with split on / in case of 9x else add 20"""
    saison = row['Saison']
    first_year, second_year = saison.split('/')
    first_year = first_year.strip()
    if first_year.startswith('9'):
        return int("19"+first_year)
    else:
        return int("20"+first_year)

In [1088]:
col9 = new_df.apply(get_year, axis=1) # get column data with an index
new_df = new_df.assign(Jahr=col9.values) # assign values to column 'Schadix_Tendenz'

In [1089]:
#new_df['year'] = new_df.apply(get_year, axis=1)

In [1090]:
def home_draw_away_generic(Htore,Gtore)->str:
    """
    A = away team won
    H = home team won
    D = draw
    """
    diff = Htore - Gtore
    if pd.isnull(Htore) or pd.isnull(Gtore):
        return np.nan
    if diff < 0:
        return 2
    elif diff > 0:
        return 1
    else:
        return 0

In [1091]:
def home_draw_away(row):
    return home_draw_away_generic(row['Htore'], row['Gtore'])

In [1092]:
def home_draw_away_uwe(row):
    return home_draw_away_generic(row['UHT'], row['UGT'])

In [1093]:
def home_draw_away_schadix(row):
    return home_draw_away_generic(row['SHT'], row['SGT'])

In [1094]:
col1 = new_df.apply(home_draw_away, axis=1) # get column data with an index
new_df = new_df.assign(Tendenz=col1.values) # assign values to column 'c'

In [1095]:
def Tend_td_vergleich(HT, GT, THT, TGT, Tendenz, Tipp_Tendenz, TD, Tipp_TD):
    
    Tendenz_diff = Tendenz - Tipp_Tendenz
    
    if pd.isnull(Tendenz) or pd.isnull(Tipp_Tendenz):
        return np.nan
    if Tendenz_diff == 0: # Tendenz stimmt
        pkt = 1 
    else:
        return 0 # Tendenz stimmt nicht
    
     
    TD_diff = TD - Tipp_TD
    
    if TD_diff == 0:  #Tordifferenz stimmt
        pkt = 1
    else:
        return 1 #Tordifferenz stimmt nicht
    
    Heimtore_diff = HT - THT
    Gasttore_diff = GT - TGT
     
    if Heimtore_diff !=0 or Gasttore_diff != 0: # Ergebnis nicht richtig, gibt aber nur 1 PKT trotz richtiger TD, weil Remis
             if Tendenz == 0:
                return 1
             else: 
                return 2    # Ergebnis nicht richtig, gibt aber 2 PKT wegen richtiger TD und kein Remis
    else:
        return 3 # Tendenz + Ergebnis genau richtig

In [1096]:
def Tendenz_TD_vergleich_uwe(row):
    return Tend_td_vergleich(row['Htore'], row['Gtore'],row['UHT'],row['UGT'], row['Tendenz'], row['Uwe_Tendenz'], row['TD'], row['TD_Uwe'])

In [1097]:
def Tendenz_TD_vergleich_schadix(row):
    return Tend_td_vergleich(row['Htore'], row['Gtore'],row['SHT'],row['SGT'], row['Tendenz'], row['Schadix_Tendenz'], row['TD'], row['TD_Schadix'])

In [1098]:
col2 = new_df.apply(tordiff, axis=1) # get column data with an index
new_df = new_df.assign(TD=col2.values) # assign values to column 'TD'

In [1099]:
col3 = new_df.apply(tordiff_Uwe, axis=1) # get column data with an index
new_df = new_df.assign(TD_Uwe=col3.values) # assign values to column 'TD'

In [1100]:
col4 = new_df.apply(tordiff_Schadix, axis=1) # get column data with an index
new_df = new_df.assign(TD_Schadix=col4.values) # assign values to column 'TD'

In [1101]:
col5 = new_df.apply(home_draw_away_uwe, axis=1) # get column data with an index
new_df = new_df.assign(Uwe_Tendenz=col5.values) # assign values to column 'Uwe_Tendenz'

In [1102]:
col6 = new_df.apply(home_draw_away_schadix, axis=1) # get column data with an index
new_df = new_df.assign(Schadix_Tendenz=col6.values) # assign values to column 'Schadix_Tendenz'

In [1103]:
col7 = new_df.apply(Tendenz_TD_vergleich_uwe, axis=1) # get column data with an index
new_df = new_df.assign(Uwe_Tendenz_Pkt=col7.values) # assign values to column 'Uwe_Tendenz_Pkt'

In [1104]:
col8 = new_df.apply(Tendenz_TD_vergleich_schadix, axis=1) # get column data with an index
new_df = new_df.assign(Schadix_Tendenz_Pkt=col8.values) # assign values to column 'Schadix_Tendenz_Pkt'

In [1105]:
df_points_sum = new_df.groupby('Jahr').sum(['Uwe_Tendenz_Pkt', 'Schadix_Tendenz_Pkt'])[['Uwe_Tendenz_Pkt', 'Schadix_Tendenz_Pkt']]

In [1106]:
df_points_sum

,Uwe_Tendenz_Pkt,Schadix_Tendenz_Pkt
Jahr,,
1995,83.0,72.0
1996,221.0,205.0
1997,211.0,197.0
1998,235.0,196.0
1999,219.0,222.0
2000,216.0,177.0
2001,221.0,192.0
2002,215.0,187.0
2003,213.0,250.0
